# Image to Text
#### The goal is to turn an image into a textual description for further processing in our "Text to Board" stage.

Older methods include using a CNN/RNN combination (encoder-decoder). The benefits for this method are more control over outputs and it can run on older hardware. However, for a more modern approach, I will be trying BLIP (Bootstrapping Language-Image Pre-training). Specifically, I will be using BLIP-2 following this guide from HuggingFace on how to deploy:
https://huggingface.co/blog/blip-2

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git